### Imports

In [1]:
from __future__ import division, print_function
from collections import OrderedDict

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models.inception import model_urls
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# import sparseconvnet.legacy as scn

import matplotlib.pyplot as plt
import time
import os
import cv2
from PIL import Image, ImageChops

plt.ion()   # interactive mode
%matplotlib inline
%env TMP="/tmp"
%env JOBLIB_TEMP_FOLDER="/tmp"
%env CUDA_VISIBLE_DEVICES=1

env: TMP="/tmp"
env: JOBLIB_TEMP_FOLDER="/tmp"
env: CUDA_VISIBLE_DEVICES=1


In [2]:
def summary(input_size, model):
    def register_hook(module):
        def hook(module, input, output):
            class_name = str(module.__class__).split('.')[-1].split("'")[0]
            module_idx = len(summary)

            m_key = '%s-%i' % (class_name, module_idx+1)
            summary[m_key] = OrderedDict()
            summary[m_key]['input_shape'] = list(input[0].size())
            summary[m_key]['input_shape'][0] = -1
            summary[m_key]['output_shape'] = list(output.size())
            summary[m_key]['output_shape'][0] = -1

            params = 0
            if hasattr(module, 'weight'):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                if module.weight.requires_grad:
                    summary[m_key]['trainable'] = True
                else:
                    summary[m_key]['trainable'] = False
            if hasattr(module, 'bias'):
                params +=  torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]['nb_params'] = params

        if not isinstance(module, nn.Sequential) and \
           not isinstance(module, nn.ModuleList) and \
           not (module == model):
            hooks.append(module.register_forward_hook(hook))

    # check if there are multiple inputs to the network
    if isinstance(input_size[0], (list, tuple)):
        x = [Variable(torch.rand(1,*in_size)) for in_size in input_size]
    else:
        x = Variable(torch.rand(1,*input_size))

    # create properties
    summary = OrderedDict()
    hooks = []
    # register hook
    model.apply(register_hook)
    # make a forward pass
    model(x)
    # remove these hooks
    for h in hooks:
        h.remove()

    return summary

### Data

In [3]:
TRAIN_DIR = '../data/boneage-training-dataset/'
TEST_DIR = '../data/boneage-validation-dataset/'
TRAIN_LABEL = '../data/train.csv'
TEST_LABEL = '../data/dataset_data_file-validation-gender-fa677c87-bd0d-44a1-92f3-f5966389798b.csv'
FILETYPE = '.png'
img_scale = 256
img_size = 244
n_channels = 3

use_gpu = torch.cuda.is_available()

### Dataset class

In [4]:
class BoneAgeDataset(Dataset):
    """Bone Age dataset."""

    def __init__(self, root_dir, labels, filetype, img_size, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
        self.filetype = filetype
        self.img_size = img_size
        self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        self.training = True if mode == 'train' else False
        self.predicting = True if mode == 'pred' else False

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
#         print(idx)
#         sample = 1.0
        img_name = os.path.join(self.root_dir, str(self.labels.iloc[idx]['id']) + self.filetype)
        img = Image.open(img_name).convert('RGB')
#         print(img_name, img.size)
        
        # randomize thresholding
        if self.training:
            img = img.point(lambda x: x if x < np.random.randint(200,220) else 0)
        else:
            img = img.point(lambda x: x if x < 220 else 0)
            
        # resizing
#         print('PIL size:', img.size)
        img = np.asarray(img)
#         print('Np size: ',img.shape)
        if img.shape[1] > img.shape[0]:
#             print(img.shape)
            img = np.swapaxes(img,0,1)
    
        img = cv2.resize(img, (img_scale,img_scale))

#         img = cv2.resize(img, (img_scale,img_scale+210))
#         img = img[210:,:]
#         print(img.shape)

        # rotation
        if self.training:
            rot_ang = range(0,360,30)
            rot_ang = rot_ang[np.random.randint(0,len(rot_ang))]
    #         print(rot_ang)
            scale = np.random.uniform(0.8,1.2)
    #         print(scale)
            rot_mat = cv2.getRotationMatrix2D((img.shape[0]/2,img.shape[1]/2), rot_ang, scale);
            img = cv2.warpAffine(img, rot_mat, (img.shape[0],img.shape[1]))
        
        # randomize thresholding
        if self.training:
            thresh_block_size = np.random.randint(5,9) // 2 * 2 + 1
            thresh_param = np.random.randint(5,9)
        else:
            thresh_block_size = 7
            thresh_param = 7
        
        img_he = self.clahe.apply(cv2.convertScaleAbs(img[:,:,0]))
        img_mask = cv2.adaptiveThreshold(img_he, 1, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,
                                         thresh_block_size, thresh_param)
        img_ch = img_he * img_mask
        
        if n_channels == 1:
            img = img_ch
        elif n_channels == 3:
            for i in xrange(img.shape[2]):
                img[:,:,i] = img_ch
        else:
            print('\n Wrong number of channels \n')

        img = Image.fromarray(np.uint8(img))
#         print('Img:', img.size)
        
#         image = cv2.imread(img_name)[:,:,::-1]
        gender = self.labels.iloc[idx]['male'].astype(np.int32)
        pat_id = self.labels.iloc[idx]['id'].astype(np.int32)
        
        if not self.predicting:
            boneage = self.labels.iloc[idx]['boneage'].astype(np.float32)
            label = np.hstack((boneage, gender, pat_id))
        else:
            label = np.hstack((gender, pat_id))
#         print(label)

        if self.transform:
            img = self.transform(img)
            
        sample = {'image': img, 'label': label}

        return sample

### Transforms

In [5]:
data_transforms = {
    'train': transforms.Compose([
#         transforms.Scale(300),
#         transforms.CenterCrop(384),
        transforms.RandomSizedCrop(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
#         transforms.Scale(300),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'pred': transforms.Compose([
#         transforms.Scale(300),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

### Data loaders

In [6]:
labels = pd.read_csv(TRAIN_LABEL, dtype=np.uint16)
labels = labels[labels['male']==0]
labels = labels.reset_index()
bins = xrange(40)
labels['bins'] = np.digitize(labels['boneage'], bins=bins)

In [7]:
skf = StratifiedKFold(n_splits=7)
# skf = KFold(n_splits=7)
splits = []
for train, test in skf.split(np.zeros(len(labels)), labels['bins']):
    fold_label = {
        'train': labels.loc[train, ('id', 'boneage', 'male')],
        'val': labels.loc[test, ('id', 'boneage', 'male')]
    }
    splits.append(fold_label)

/home/user/software/miniconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=7.
  % (min_groups, self.n_splits)), Warning)


In [8]:
# splits[0]['train']['boneage'].hist()

In [9]:
# splits[0]['val']['boneage'].hist()

In [10]:
fold_label = splits[0]

In [11]:
image_datasets = {x: BoneAgeDataset(TRAIN_DIR, fold_label[x], FILETYPE, img_size, data_transforms[x], x)
                  for x in ['train', 'val']}
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64, shuffle=True, num_workers=12, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print(dataset_sizes)

{'train': 4944, 'val': 834}


In [12]:
for i_batch, sample in enumerate(dataloders['val']):
    print(i_batch, sample['image'].size())
    print(i_batch, sample['image'].mean())
#     for i in xrange(sample['image'].size()[0]):
#         img = sample['image'][i,0,:,:].numpy()
#         plt.figure()
#         plt.imshow(img, cmap=plt.cm.gray)
    if i_batch == 0:
        break

0 torch.Size([64, 3, 244, 244])
0 0.232455238896


### Training

In [13]:
model_urls['inception_v3_google'] = model_urls['inception_v3_google'].replace('https://', 'http://')

class Net(nn.Module):
    def __init__(self, input_shape=(n_channels, img_size, img_size)):
        super(Net, self).__init__()
        
#         # inception_v3
#         self.bn = nn.BatchNorm2d(input_shape[0])
#         resnet = models.inception_v3(pretrained=True)
        
#         num_ftrs = resnet.fc.in_features
#         resnet.fc = nn.Linear(num_ftrs, 1)
        
#         num_ftrs_aux = resnet.AuxLogits.fc.in_features
#         resnet.AuxLogits.fc = nn.Linear(num_ftrs_aux, 1)
        
#         self.resnet = resnet
        
        # resnet
#         self.bn = nn.BatchNorm2d(input_shape[0])
        resnet = models.resnet50(pretrained=True)
        for param in resnet.parameters():
            param.requires_grad = False
        num_ftrs = resnet.fc.in_features
        boneage_clf = nn.Sequential(
            nn.Dropout(),
            nn.Linear(num_ftrs, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 1)
        )
        resnet.fc = boneage_clf
        self.resnet = resnet

#         self.net = nn.Sequential(
#             nn.Conv2d(input_shape[0], 192, 5, stride=3),
#             nn.BatchNorm2d(192),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(192, 160, 1, stride=1),
#             nn.BatchNorm2d(160),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(160, 96, 1, stride=1),
#             nn.BatchNorm2d(96),
#             nn.ReLU(inplace=True),
            
#             nn.MaxPool2d(3, stride=2, ceil_mode=True),
#             nn.Dropout(),
            
#             nn.Conv2d(96, 192, 5, stride=2),
#             nn.BatchNorm2d(192),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(192, 192, 1, stride=1),
#             nn.BatchNorm2d(192),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(192, 192, 1, stride=1),
#             nn.BatchNorm2d(192),
#             nn.ReLU(inplace=True),
            
#             nn.AvgPool2d(3, stride=2, ceil_mode=True),
#             nn.Dropout(),
        
#             nn.Conv2d(192, 192, 3),
#             nn.BatchNorm2d(192),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(192, 192, 1, stride=1),
#             nn.BatchNorm2d(192),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(192, 1, 1, stride=1),
#             nn.BatchNorm2d(1),
#             nn.ReLU(inplace=True)
                                 
# #             nn.AvgPool2d(12, stride=12, ceil_mode=True)
#         )
#         self.boneage_clf = nn.Sequential(
#             nn.Dropout(),
#             nn.Linear(32*32, 512),
#             nn.ReLU(inplace=True),
#             nn.Dropout(),
#             nn.Linear(512, 128),
#             nn.ReLU(inplace=True),
#             nn.Linear(128, 1)
#         )
#         self.gender_clf = nn.Sequential(
#             nn.Dropout(),
#             nn.Linear(32*32, 512),
#             nn.ReLU(inplace=True),
#             nn.Dropout(),
#             nn.Linear(512, 128),
#             nn.ReLU(inplace=True),
#             nn.Linear(128, 1)
#         )
        
#         resnet.avgpool = nn.AdaptiveAvgPool2d(2)
#         num_ftrs = net.fc.in_features
#         net.fc = nn.Linear(num_ftrs, 1)
#         self.net = net

    def forward(self, x):

        x = self.resnet(x)
        
#         x = self.net(self.bn(x))
#         x = self.net(x)
#         print(x.size())
#         x = x.view(-1, 32*32)
#         g_x = self.gender_clf(x)
#         ba_x = self.boneage_clf(x)
#         x = F.relu(self.lin128(F.dropout(x)))
#         x = self.lin1(F.dropout(x))
#         x = self.resnet(self.bn(x))
#         return ba_x
#         return ba_x, g_x
        return x
    
model = Net([n_channels, img_size, img_size])
    
# print(summary([3, 640, 640], model))
print(model)

if use_gpu:
    model.cuda()

Net (
  (resnet): ResNet (
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (relu): ReLU (inplace)
    (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    (layer1): Sequential (
      (0): Bottleneck (
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
        (relu): ReLU (inplace)
        (downsample): Sequential (
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(256, eps=1e-05, mome

In [14]:
model.load_state_dict(torch.load('../models/10-4-2000-resnet50-0fold7-fem-02-02.pth'))

In [15]:
lr = 0.001
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.resnet.fc.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [16]:
def pearsonr(x, y):
    x = x.view(-1)
    mean_x = torch.mean(x)
    mean_y = torch.mean(y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    r_num = xm.dot(ym)
    r_den = torch.norm(xm, 2) * torch.norm(ym, 2)
    r_val = r_num / r_den
    return r_val

def ccc(x, y):
#     print('X:', x)
    mean_x = torch.mean(x)
#     print('Mean x:', mean_x)
    mean_y = torch.mean(y)
#     print('Mean y:', mean_y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    xn = torch.norm(xm, 2)
#     print('Xn:', xn)
    yn = torch.norm(ym, 2)
#     print('Yn:', yn)
    xydot = ym.dot(xm.view(-1))
#     print('XYdot:', xydot)

    ccc_num = 2 * xydot / xm.size()[0]
#     print('CCC num:', ccc_num)
    ccc_den = xn + yn + torch.pow(mean_x.sub(mean_y), 2)
#     print('CCC den:', ccc_den)

    ccc = ccc_num / ccc_den
    return ccc
    

def train(epoch, train_loader):
    since = time.time()
    exp_lr_scheduler.step()
    model.train()
    
    train_boneage_loss = 0
    train_gender_loss = 0
    train_loss = 0
    train_ccc = 0
    num_samp = len(train_loader.dataset)
    
    for batch_idx, sample in enumerate(train_loader):
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        
        ba_output = model(data)
        
        # 2 heads MTL output
#         ba_output, g_output = model(data)
#         print(output1)
#         print(output2)
        
#         for i in xrange(output.size[0]):
#             if output[i,1] > 0.5:
#                 output[i,0] = (output[i,0] // 6) * 6
#             else:
#                 output[i,0] = (output[i,0] // 2) * 2

#         boneage_out = torch.round(output[:,0].div(6)).mul(6)
#         boneage_out = output[:,0]

#         boneage_loss = F.l1_loss(output.view(-1), target[:,0])
#         train_boneage_loss += boneage_loss.data[0]
        boneage_loss = F.smooth_l1_loss(ba_output.view(-1), target[:,0])
        train_boneage_loss += boneage_loss.data[0]
        
#         gender_loss = F.binary_cross_entropy_with_logits(g_output.view(-1), target[:,1])
#         train_gender_loss += gender_loss.data[0]
        
#         loss = 0.75 * boneage_loss + 0.25 * gender_loss
        loss = boneage_loss
#         loss = gender_loss
#         loss = F.l1_loss(output1.view(-1), target[:,0]) + F.l1_loss(output2.view(-1), target[:,0])
#         loss = 0.8 * F.l1_loss(output.view(-1), target[:,0]) + 0.2 * F.binary_cross_entropy(F.sigmoid(output.view(-1)), target[:,1])
#         loss = F.l1_loss(output.view(-1), target[:,0])
        
        train_loss += loss.data[0]
#         c = ccc(boneage_out, target[:,0])
        c = ccc(ba_output, target[:,0])
        train_ccc += c.data.cpu().numpy()[0]
        
        loss.backward()
        optimizer.step()
#         if batch_idx % (1000 // len(target)) == 0:
#     print('Train ep: {} [{}/{} ({:.0f}%)]\tBoneAge Loss: {:.4f}\tGender Loss: {:.4f}\t Loss: {:.4f}\tCCC: {:.4f}'.format(
#         epoch, batch_idx * len(data), num_samp,
#         100. * batch_idx / num_samp, train_boneage_loss / num_samp,
#         train_gender_loss / num_samp, train_loss / num_samp,
#         train_ccc / num_samp))
    
    print('\nTrain Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.4f}\tCCC: {:.4f}'.format(
        epoch, batch_idx * len(data), num_samp,
        100. * batch_idx / num_samp, train_loss / num_samp, train_ccc / num_samp))
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

def test(test_loader):
    since = time.time()
    model.eval()
    
    test_avg_gender = 0
    test_boneage_loss = 0
    test_boneage_l1_loss = 0
    test_boneage_round_loss = 0
    test_boneage_round_loss_2 = 0
    test_boneage_round_loss_6 = 0
    test_gender_loss = 0
    test_loss = 0
    test_ccc = 0
    test_pearsonr = 0
    
    num_samp = len(test_loader.dataset)
    for sample in test_loader:
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)

#         test_avg_gender += sample['label'][:,1].sum()
        ba_output = model(data)
        
        # 2 heads MTL output
#         ba_output, g_output = model(data)
#         print(ba_output)
        
#         print(output.size())
#         boneage_out = torch.round(output[:,0].div(6)).mul(6)
#         boneage_out = output[:,0]
#         print(boneage_out)
#         print(target[:,0])
#         boneage_loss = F.l1_loss(output.view(-1), target[:,0])

        boneage_loss = F.smooth_l1_loss(ba_output.view(-1), target[:,0])
        boneage_l1_loss = F.l1_loss(ba_output.view(-1), target[:,0])
# #         print(boneage_loss)
        boneage_round_loss = F.l1_loss(torch.round(ba_output.view(-1)), target[:,0])
# #         print(boneage_round_loss)
        boneage_round_loss_2 = F.l1_loss(torch.round(ba_output.view(-1).div(2)).mul(2), target[:,0])
# #         print(boneage_round_loss_2)
        boneage_round_loss_6 = F.l1_loss(torch.round(ba_output.view(-1).div(6)).mul(6), target[:,0])
# #         print(boneage_round_loss_6)
        
        test_boneage_loss += boneage_loss.data[0]
        test_boneage_l1_loss += boneage_l1_loss.data[0]
        test_boneage_round_loss += boneage_round_loss.data[0]
        test_boneage_round_loss_2 += boneage_round_loss_2.data[0]
        test_boneage_round_loss_6 += boneage_round_loss_6.data[0]
    
#         print(test_boneage_loss)
#         gender_loss = F.binary_cross_entropy_with_logits(g_output.view(-1), target[:,1])
#         test_gender_loss += gender_loss.data[0]

#         loss = 0.75 * boneage_loss + 0.25 * gender_loss
        loss = boneage_loss
#         loss = 0.8 * F.l1_loss(output.view(-1), target[:,0]) + 0.2 * F.binary_cross_entropy(F.sigmoid(output.view(-1)), target[:,1])
#         loss = F.l1_loss(output.view(-1), target[:,0])

        test_loss += loss.data[0]
#         c = ccc(boneage_out, target[:,0])
        c = ccc(ba_output, target[:,0])
        r = pearsonr(ba_output, target[:,0])
        test_ccc += c.data.cpu().numpy()[0]
        test_pearsonr += r.data.cpu().numpy()[0]

    print('\nTest set: BoneAge loss: {:.4f}, loss: {:.4f}, CCC: {:.4f}, R: {:.4f}'.format(
        test_boneage_loss / num_samp, test_loss / num_samp, test_ccc / num_samp, test_pearsonr / num_samp))
    print('Test set: L1 loss: {:.4f},Round loss: {:.4f}, Round2 loss: {:.4f}, Round6: {:.4f}, Gender: {:.4f}'.format(
        test_boneage_l1_loss / num_samp, test_boneage_round_loss / num_samp, test_boneage_round_loss_2 / num_samp,
        test_boneage_round_loss_6 / num_samp, test_avg_gender / num_samp))
    
#     print('\nTest set: Avg loss: {:.4f} CCC: {:.4f}\n'.format(
#         test_loss / num_samp, test_ccc / num_samp))
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s\n'.format(time_elapsed // 60, time_elapsed % 60))
    
    return test_boneage_l1_loss / num_samp

In [ ]:
# best_loss = 100
for epoch in range(1, 4 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

In [ ]:
for param in model.parameters():
    param.requires_grad = True
lr = 0.001
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [ ]:
for epoch in range(1, 60 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

In [ ]:
# resuming train

best_loss = test(dataloders['val'])
best_model_wts = model.state_dict()

for param in model.parameters():
    param.requires_grad = True

lr = 0.0001
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

for epoch in range(1, 60 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            epoch_val_loss = test(dataloders[phase])
            if epoch_val_loss < best_loss:
                best_loss = epoch_val_loss
                best_model_wts = model.state_dict()
                print('Saved model weights.')


Test set: BoneAge loss: 0.3529, loss: 0.3529, CCC: 0.0481, R: 0.0146
Test set: L1 loss: 0.3611,Round loss: 0.3610, Round2 loss: 0.3612, Round6: 0.3618, Gender: 0.0000
Training complete in 0m 8s


Train Epoch: 1 [1232/4944 (2%)]	 Loss: 0.2127	CCC: 0.0586
Training complete in 1m 1s

Test set: BoneAge loss: 0.2492, loss: 0.2492, CCC: 0.0726, R: 0.0149
Test set: L1 loss: 0.2575,Round loss: 0.2574, Round2 loss: 0.2572, Round6: 0.2545, Gender: 0.0000
Training complete in 0m 8s

Saved model weights.

Train Epoch: 2 [1232/4944 (2%)]	 Loss: 0.2068	CCC: 0.0586
Training complete in 1m 2s

Test set: BoneAge loss: 0.2309, loss: 0.2309, CCC: 0.1016, R: 0.0150
Test set: L1 loss: 0.2392,Round loss: 0.2398, Round2 loss: 0.2400, Round6: 0.2401, Gender: 0.0000
Training complete in 0m 7s

Saved model weights.

Train Epoch: 3 [1232/4944 (2%)]	 Loss: 0.2050	CCC: 0.0596
Training complete in 1m 3s

Test set: BoneAge loss: 0.2296, loss: 0.2296, CCC: 0.0610, R: 0.0127
Test set: L1 loss: 0.2379,Round loss: 0.23


Train Epoch: 29 [1232/4944 (2%)]	 Loss: 0.1785	CCC: 0.0625
Training complete in 1m 1s

Test set: BoneAge loss: 0.2154, loss: 0.2154, CCC: 0.0897, R: 0.0153
Test set: L1 loss: 0.2237,Round loss: 0.2234, Round2 loss: 0.2246, Round6: 0.2233, Gender: 0.0000
Training complete in 0m 7s


Train Epoch: 30 [1232/4944 (2%)]	 Loss: 0.1763	CCC: 0.0634
Training complete in 1m 1s

Test set: BoneAge loss: 0.2103, loss: 0.2103, CCC: 0.0748, R: 0.0153
Test set: L1 loss: 0.2184,Round loss: 0.2183, Round2 loss: 0.2174, Round6: 0.2196, Gender: 0.0000
Training complete in 0m 7s


Train Epoch: 31 [1232/4944 (2%)]	 Loss: 0.1817	CCC: 0.0622
Training complete in 1m 2s

Test set: BoneAge loss: 0.2177, loss: 0.2177, CCC: 0.0899, R: 0.0153
Test set: L1 loss: 0.2260,Round loss: 0.2262, Round2 loss: 0.2265, Round6: 0.2263, Gender: 0.0000
Training complete in 0m 8s


Train Epoch: 32 [1232/4944 (2%)]	 Loss: 0.1811	CCC: 0.0628
Training complete in 1m 3s

Test set: BoneAge loss: 0.2116, loss: 0.2116, CCC: 0.0711, R: 0

In [19]:
lr = 0.00001
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

for epoch in range(1, 60 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            epoch_val_loss = test(dataloders[phase])
            if epoch_val_loss < best_loss:
                best_loss = epoch_val_loss
                best_model_wts = model.state_dict()
                print('Saved model weights.')


Train Epoch: 1 [1232/4944 (2%)]	 Loss: 0.1759	CCC: 0.0628
Training complete in 1m 2s

Test set: BoneAge loss: 0.2150, loss: 0.2150, CCC: 0.0701, R: nan
Test set: L1 loss: 0.2233,Round loss: 0.2229, Round2 loss: 0.2235, Round6: 0.2222, Gender: 0.0000
Training complete in 0m 6s


Train Epoch: 2 [1232/4944 (2%)]	 Loss: 0.1794	CCC: 0.0628
Training complete in 1m 1s

Test set: BoneAge loss: 0.2002, loss: 0.2002, CCC: 0.0767, R: 0.0154
Test set: L1 loss: 0.2085,Round loss: 0.2085, Round2 loss: 0.2090, Round6: 0.2075, Gender: 0.0000
Training complete in 0m 7s


Train Epoch: 3 [1232/4944 (2%)]	 Loss: 0.1761	CCC: 0.0627
Training complete in 1m 1s

Test set: BoneAge loss: 0.2054, loss: 0.2054, CCC: 0.0792, R: 0.0154
Test set: L1 loss: 0.2136,Round loss: 0.2140, Round2 loss: 0.2126, Round6: 0.2143, Gender: 0.0000
Training complete in 0m 7s


Train Epoch: 4 [1232/4944 (2%)]	 Loss: 0.1804	CCC: 0.0622
Training complete in 1m 2s

Test set: BoneAge loss: 0.2026, loss: 0.2026, CCC: 0.0844, R: 0.0153
T


Train Epoch: 30 [1232/4944 (2%)]	 Loss: 0.1713	CCC: 0.0645
Training complete in 1m 1s

Test set: BoneAge loss: 0.2221, loss: 0.2221, CCC: 0.0712, R: 0.0153
Test set: L1 loss: 0.2304,Round loss: 0.2300, Round2 loss: 0.2292, Round6: 0.2274, Gender: 0.0000
Training complete in 0m 6s


Train Epoch: 31 [1232/4944 (2%)]	 Loss: 0.1727	CCC: 0.0634
Training complete in 1m 1s

Test set: BoneAge loss: 0.2134, loss: 0.2134, CCC: 0.0896, R: 0.0153
Test set: L1 loss: 0.2216,Round loss: 0.2213, Round2 loss: 0.2209, Round6: 0.2193, Gender: 0.0000
Training complete in 0m 6s


Train Epoch: 32 [1232/4944 (2%)]	 Loss: 0.1775	CCC: 0.0632
Training complete in 1m 1s

Test set: BoneAge loss: 0.2098, loss: 0.2098, CCC: 0.0708, R: 0.0130
Test set: L1 loss: 0.2180,Round loss: 0.2179, Round2 loss: 0.2182, Round6: 0.2163, Gender: 0.0000
Training complete in 0m 7s


Train Epoch: 33 [1232/4944 (2%)]	 Loss: 0.1754	CCC: 0.0642
Training complete in 1m 1s

Test set: BoneAge loss: 0.2097, loss: 0.2097, CCC: 0.0748, R: 0


Train Epoch: 59 [1232/4944 (2%)]	 Loss: 0.1737	CCC: 0.0643
Training complete in 1m 1s

Test set: BoneAge loss: 0.2057, loss: 0.2057, CCC: 0.0714, R: 0.0153
Test set: L1 loss: 0.2140,Round loss: 0.2140, Round2 loss: 0.2141, Round6: 0.2114, Gender: 0.0000
Training complete in 0m 6s


Train Epoch: 60 [1232/4944 (2%)]	 Loss: 0.1731	CCC: 0.0642
Training complete in 1m 2s

Test set: BoneAge loss: 0.2043, loss: 0.2043, CCC: 0.0782, R: 0.0154
Test set: L1 loss: 0.2126,Round loss: 0.2123, Round2 loss: 0.2127, Round6: 0.2136, Gender: 0.0000
Training complete in 0m 6s



In [ ]:
lr = 0.0001
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in range(1, 60 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            epoch_val_loss = test(dataloders[phase])
            if epoch_val_loss < best_loss:
                best_loss = epoch_val_loss
                best_model_wts = model.state_dict()
                print('Saved model weights.')


Train Epoch: 1 [1232/4944 (2%)]	 Loss: 0.1797	CCC: 0.0632
Training complete in 1m 0s

Test set: BoneAge loss: 0.2104, loss: 0.2104, CCC: 0.0735, R: 0.0152
Test set: L1 loss: 0.2186,Round loss: 0.2188, Round2 loss: 0.2178, Round6: 0.2176, Gender: 0.0000
Training complete in 0m 7s


Train Epoch: 2 [1232/4944 (2%)]	 Loss: 0.1778	CCC: 0.0623
Training complete in 1m 2s

Test set: BoneAge loss: 0.2049, loss: 0.2049, CCC: 0.0839, R: 0.0153
Test set: L1 loss: 0.2131,Round loss: 0.2131, Round2 loss: 0.2132, Round6: 0.2137, Gender: 0.0000
Training complete in 0m 7s


Train Epoch: 3 [1232/4944 (2%)]	 Loss: 0.1778	CCC: 0.0634
Training complete in 1m 2s

Test set: BoneAge loss: 0.2091, loss: 0.2091, CCC: 0.0773, R: 0.0154
Test set: L1 loss: 0.2173,Round loss: 0.2173, Round2 loss: 0.2173, Round6: 0.2156, Gender: 0.0000
Training complete in 0m 7s


Train Epoch: 4 [1232/4944 (2%)]	 Loss: 0.1770	CCC: 0.0633
Training complete in 1m 2s

Test set: BoneAge loss: 0.2112, loss: 0.2112, CCC: 0.0995, R: 0.015

In [ ]:
lr = 0.00001
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

for epoch in range(1, 35 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            epoch_val_loss = test(dataloders[phase])
            if epoch_val_loss < best_loss:
                best_loss = epoch_val_loss
                best_model_wts = model.state_dict()
                print('Saved model weights.')

#### save model

In [ ]:
torch.save(best_model_wts, '../models/10-4-2000-resnet50-0fold7-fem-02-02.pth')

### Prediction

In [ ]:
model.load_state_dict(torch.load('../models/10-3-0320-nin-0fold-083-116.pth'))

In [ ]:
def predict(predict_loader):
    model.eval()
    pred_gender_loss = 0
    predictions = []
    
    num_samp = len(predict_loader.dataset)
    for sample in predict_loader:
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)

        ba_output, g_output = model(data)
    
#         print(test_boneage_loss)
        gender_loss = F.binary_cross_entropy_with_logits(g_output.view(-1), target[:,0])
        pred_gender_loss += gender_loss.data[0]
        
        pred_labels = target[:,1].data.cpu().numpy()
#         print(pred_labels.shape)
        pred_bonage = ba_output.view(-1).data.cpu().numpy()
#         print(pred_bonage.shape)
        
        predictions.append(np.column_stack((pred_labels, pred_bonage)))

    print('\nPrediction Gender loss: {:.4f}'.format(pred_gender_loss / num_samp))
    
    return predictions

In [ ]:
pred_labels = pd.read_csv(TEST_LABEL, dtype=int)
pred_labels.describe()

In [ ]:
pred_image_dataset = BoneAgeDataset(TEST_DIR, pred_labels, FILETYPE, img_size, data_transforms['pred'], 'pred')
pred_dataloder = torch.utils.data.DataLoader(pred_image_dataset, batch_size=32, shuffle=True, num_workers=24,
                                             pin_memory=True)
pred_dataset_size = len(pred_image_dataset)

In [ ]:
for i_batch, sample in enumerate(pred_dataloder):
    print(i_batch, sample['image'].size())
    print(i_batch, sample['image'].mean())
#     for i in xrange(sample['image'].size()[0]):
#         img = sample['image'][i,0,:,:].numpy()
#         plt.figure()
#         plt.imshow(img, cmap=plt.cm.gray)
    if i_batch == 2:
        break

In [ ]:
preds = predict(pred_dataloder)

In [ ]:
all_preds_df = pd.DataFrame(np.vstack(tuple(preds)), columns=('id', 'boneage')).sort_values('id')
all_preds_df['id'] = all_preds_df['id'].astype(int)

In [ ]:
all_preds_df.head()

In [ ]:
all_preds_df.tail()

In [ ]:
all_preds_df.to_csv('../sub/10-3-0707-nin-0fold-083-116.csv', index=False)